In [220]:
import pandas as pd
import numpy as np

from pathlib import Path

In [221]:
data_path = Path.cwd().parent.joinpath("train_data_fix_concat.csv")

In [222]:
data = pd.read_csv(data_path.__str__())
data = data.drop_duplicates()
data = data[data['oil_property_param_value'].notna()]

### Тип масла - Масло - свойство - компоненты - свойства компонентов

In [223]:
# Все возможные масла
total_oil_ids = data["blend_id"].drop_duplicates().to_list()
print(f"Total oil ids -- {len(total_oil_ids)}")

# Все используемые компоненты
total_components = list(set(data["component_name"].to_list()))
print(f"Total chemical components -- {len(total_components)}")

# Все возможные свойства компонентов
total_component_properties = list(set(data["component_property_param_title"]))
print(f"Total chemical components properties -- {len(total_component_properties)}")

# Все возможные расширения для компонентов
total_component_classes = list(set(data["component_class"]))
print(f"Total chemical components classes -- {len(total_component_classes)}")



print(f"X vector length -- {len(total_components) * len(total_component_properties)} + N of oil addons")

Total oil ids -- 348
Total chemical components -- 112
Total chemical components properties -- 63
Total chemical components classes -- 14
X vector length -- 7056 + N of oil addons


In [224]:
# Фиксируем только свойство по ТЗ
workdata = data.loc[data["oil_property_param_title"] == "ad7e6027-00b8-4c27-918c-d1561f949ad8"]
workdata_oil_ids = list(set(workdata["blend_id"].to_list()))

In [225]:
workdata

,oil_type,blend_id,oil_property_param_title,oil_property_param_value,component_name,component_class,polymer,component_property_param_title,component_property_param_value,smiles
358,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,ad7e6027-00b8-4c27-918c-d1561f949ad8,103300.0,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,02236ee6-5eec-4368-a2e4-6f2e73fb0f96,0.0000,CCCCC
359,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,ad7e6027-00b8-4c27-918c-d1561f949ad8,103300.0,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,2511714c-ab50-4566-bc92-8e4095d87d01,0.0000,CCCCC
360,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,ad7e6027-00b8-4c27-918c-d1561f949ad8,103300.0,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,38b06d9a-bcf2-40de-8e1b-9c3988c42406,0.0001,CCCCC
361,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,ad7e6027-00b8-4c27-918c-d1561f949ad8,103300.0,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,f216a0c7-d453-4b15-9b4a-7647cbe2d874,0.0990,CCCCC
362,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,ad7e6027-00b8-4c27-918c-d1561f949ad8,103300.0,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,9703e283-f529-4fdb-8d84-24413b2b4338,0.0001,CCCCC
...,...,...,...,...,...,...,...,...,...,...
359360,60c1209c-8e77-467e-a01f-b6321801a260,45f1e44a-9410-11ee-8abf-005056921581,ad7e6027-00b8-4c27-918c-d1561f949ad8,12510.0,1433126f-b36a-4e69-ad2f-ee12d8b46275,NaN,NaN,266a1a34-14d1-49f7-b1ca-d354f9ff6ce8,NaN,NaN
359361,60c1209c-8e77-467e-a01f-b6321801a260,45f1e44a-9410-11ee-8abf-005056921581,ad7e6027-00b8-4c27-918c-d1561f949ad8,12510.0,1433126f-b36a-4e69-ad2f-ee12d8b46275,NaN,NaN,2a6e6683-e8f2-49ae-8114-b752a1e97c60,NaN,NaN
359363,60c1209c-8e77-467e-a01f-b6321801a260,45f1e44a-9410-11ee-8abf-005056921581,ad7e6027-00b8-4c27-918c-d1561f949ad8,12510.0,1433126f-b36a-4e69-ad2f-ee12d8b46275,NaN,NaN,f400be8b-3bed-4f17-8a12-9fabd9b73586,NaN,NaN
359364,60c1209c-8e77-467e-a01f-b6321801a260,45f1e44a-9410-11ee-8abf-005056921581,ad7e6027-00b8-4c27-918c-d1561f949ad8,12510.0,1433126f-b36a-4e69-ad2f-ee12d8b46275,NaN,NaN,1931ea10-9783-4065-a77c-04532d7d0559,NaN,NaN


In [232]:
values = []
for component in total_components:
    value = len(list(set(workdata.loc[workdata["component_name"] == component]["component_property_param_title"].to_list())))
    print(value)

18
1
21
1
21
18
18
18
18
18
21
22
18
21
18
18
18
21
18
18
21
18
34
18
18
18
18
21
18
18
18
18
18
18
21
18
18
0
18
18
18
19
22
21
21
18
21
21
18
18
18
21
18
18
24
18
18
18
18
4
20
1
18
18
18
18
18
18
18
18
18
18
21
18
0
21
21
18
18
20
23
18
18
18
18
18
21
18
18
1
18
21
18
18
1
21
18
33
0
22
21
18
18
18
18
19
18
18
18
18
21
18


### Создаем шаблон вектора X

In [312]:
vector_template = np.zeros(len(total_components) * len(total_component_properties))

def convertDataToVector(components, components_propeties_names, components_propetries_values):
    result_vector = np.zeros(len(total_components) * len(total_component_properties))
    result_vector = np.array([np.float64(-1.0) for x in result_vector])
    
    for component_index, component_name in enumerate(components):
        
        # Индекс в векторе конкретного компонента
        component_vector_index = total_components.index(component_name)
        
        for component_property_index, component_propetry in enumerate(components_propeties_names[component_index]):

            # Индекс в векторе конкретного свойства компонента
            property_vector_index = total_component_properties.index(component_propetry)

            inserting_index = int((component_vector_index *  len(total_component_properties)) + property_vector_index)
            result_vector[inserting_index] = components_propetries_values[component_index][component_property_index]
            
    return result_vector

### 1) Кодируем масло в вектор компонент

In [313]:
Y_data = []
X_data = []

In [314]:
current_id = workdata_oil_ids[1]

for index in range(len(workdata_oil_ids)):
    current_id = workdata_oil_ids[index]
    current_oil_data = workdata.loc[workdata["blend_id"] == current_id]
    
    components_in_oil = list(set(current_oil_data["component_name"].to_list()))
    
    components_propetries_to_function = []
    components_values_to_function = []
    
    for component in components_in_oil:
        # Извлекаю данные
        current_component_oil_data = current_oil_data.loc[current_oil_data["component_name"] == component]
        components_addons_class = list(set(current_component_oil_data["component_class"].to_list()))
        components_addons_polymer = list(set(current_component_oil_data["polymer"].to_list()))
        components_properties = current_component_oil_data["component_property_param_title"].to_list()
    
        # Подготавливаю для вектора
        current_component_properties = []
        current_component_values = []
    
        # Забираю значения по свойствам
        for component_property in components_properties:
            current_property_component_oil_data = current_component_oil_data.loc[current_component_oil_data["component_property_param_title"] == component_property]
            value = current_property_component_oil_data["component_property_param_value"].fillna(0.0).values[0]
            
            current_component_properties.append(component_property)
            current_component_values.append(value)
    
        # Добавляю по свойствам для функции векторизации
        components_values_to_function.append(current_component_values)
        components_propetries_to_function.append(current_component_properties)

    current_result_vector = convertDataToVector(components_in_oil, components_propetries_to_function, components_values_to_function)

    X_data.append(current_result_vector)
    Y_data.append(list(set(current_oil_data["oil_property_param_value"].to_list()))[0])

### 2) Формируем набор данных для обучения

In [315]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3)

### 3) Обучаем разные модели, смотрим по МАЕ какая лучше

In [316]:
from sklearn.linear_model import PassiveAggressiveRegressor

kostyan = PassiveAggressiveRegressor(max_iter=100, random_state=0)
kostyan.fit(x_train, y_train)

PassiveAggressiveRegressor(max_iter=100, random_state=0)

In [317]:
from sklearn.linear_model import Perceptron

perceptron = Perceptron(tol=1e-3, random_state=0)
perceptron.fit(x_train, y_train)

Perceptron()

In [318]:
from sklearn.linear_model import HuberRegressor

huber = HuberRegressor()
huber.fit(x_train, y_train)

C:\Users\Undefined\Documents\GitHub\Neftecode-2-Shisha-Revenge\venv\lib\site-packages\sklearn\linear_model\_huber.py:342: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



HuberRegressor()

In [319]:
from sklearn.linear_model import TheilSenRegressor

theilsen = TheilSenRegressor(random_state=0)
theilsen.fit(x_train, y_train)

TheilSenRegressor(random_state=0)

In [320]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor(max_iter=1000, tol=1e-3)
sgd.fit(x_train, y_train)

C:\Users\Undefined\Documents\GitHub\Neftecode-2-Shisha-Revenge\venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1575: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.



SGDRegressor()

In [321]:
from sklearn.ensemble import GradientBoostingRegressor

gradient_boosting = GradientBoostingRegressor(random_state=0)
gradient_boosting.fit(x_train, y_train)

GradientBoostingRegressor(random_state=0)

In [322]:
from sklearn.kernel_ridge import KernelRidge

kernel_ridge = KernelRidge(alpha=1.0)
kernel_ridge.fit(x_train, y_train)

KernelRidge(alpha=1.0)

In [323]:
from sklearn.svm import SVR

svr = SVR()
svr.fit(x_train, y_train)

SVR()

In [324]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
knn.fit(x_train, y_train)

knn_neighbors_10 = KNeighborsRegressor(n_neighbors=10)
knn_neighbors_10.fit(x_train, y_train)

knn_neighbors_2 = KNeighborsRegressor(n_neighbors=2)
knn_neighbors_2.fit(x_train, y_train)

KNeighborsRegressor(n_neighbors=2)

In [325]:
from sklearn.gaussian_process import GaussianProcessRegressor

gpr = GaussianProcessRegressor()
gpr.fit(x_train, y_train)

GaussianProcessRegressor()

In [326]:
from sklearn import tree

DT = tree.DecisionTreeRegressor()
DT.fit(x_train, y_train)

DecisionTreeRegressor()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_200 = RandomForestRegressor(n_estimators=200)
rf_200.fit(x_train, y_train)

rf_200_md_10 = RandomForestRegressor(n_estimators=200, max_depth=10)
rf_200_md_10.fit(x_train, y_train)

rf_200_md_50 = RandomForestRegressor(n_estimators=200, max_depth=50)
rf_200_md_50.fit(x_train, y_train)

rf_2000_md_500 = RandomForestRegressor(n_estimators=2000, max_depth=500)
rf_2000_md_500.fit(x_train, y_train)

In [ ]:
clfs = [kostyan, perceptron, huber, theilsen, sgd, gradient_boosting, kernel_ridge, svr, knn, knn_neighbors_10, knn_neighbors_2, gpr, DT, rf_200, rf_200_md_10, rf_200_md_50, rf_2000_md_500]

In [ ]:
names = ["kostyan", "perceptron", "huber", "theilsen", "sgd", "gradient_boosting", "kernel_ridge", "svr", "knn", "knn_neighbors_10", "knn_neighbors_2", "gpr", "DT", "rf_200", "rf_200_md_10", "rf_200_md_50", "rf_2000_md_500"]

In [ ]:
from sklearn.metrics import mean_absolute_error

mae_values = []

for index, clf in enumerate(clfs):
    value = mean_absolute_error(y_test, clf.predict(x_test))
    mae_values.append(value)

### 4) Сравнение алгоритмов

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
font = {'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)

fig, ax = plt.subplots()
bar_container = ax.bar(names, [x / 1000 for x in mae_values])
ax.set(ylabel='MAE в тысячах', title='Vectors classification', ylim=(0, 40))
plt.rcParams['figure.figsize'] = [20, 20]
plt.xticks(rotation = 90)

fig.savefig("result.png")

### 5) Предсказываем на сабмит

In [ ]:
path_to_test_data = Path().cwd().parent.joinpath("test_set_public.csv")

test_df = pd.read_csv(path_to_test_data.__str__())
test_df

In [206]:
X_test_vector = []
Ids = []

In [207]:
workdata = test_df.drop_duplicates()
workdata_oil_ids = workdata["blend_id"].drop_duplicates().to_list()
workdata_oil_ids

['d272c9a6-3332-11ed-9685-005056921581',
 'dea007ac-a065-11ed-9f24-005056921581',
 '14efbc8a-9090-11ec-83a1-005056921581',
 'e48772c8-4e07-11ed-9a6d-005056921581',
 '72b7aafa-d791-11ed-8145-005056921581',
 'cf7e06d0-bc99-11ec-9ebf-005056921581',
 'c4af79fc-5a8f-11ed-96db-005056921581',
 '53692f3c-8841-11ee-9dd6-005056921581',
 'bece165e-53fd-11ec-92cf-005056921581',
 'b617ecdc-c4a6-11ec-9ebf-005056921581',
 '4d436672-7ae6-11ed-9727-005056921581',
 '39ec78c0-637a-11ee-92b6-005056921581',
 '03ac1c22-6af9-11ed-8d22-005056921581',
 'ed0f4c7e-9590-11ed-84b7-005056921581',
 'b5e28684-a9dd-11ec-9d4e-005056921581',
 '3545c644-5441-11ed-993d-005056921581',
 'f1f40fea-2eac-11ed-8c6c-005056921581',
 '7441fe6e-4c48-11ec-8a77-005056921581',
 'ad063bfc-e4c7-11ed-9764-005056921581',
 '96f82750-0f31-11ee-9c71-005056921581',
 'e25e1804-6977-11ed-85f4-005056921581',
 'c16c5606-028e-11ed-9242-005056921581',
 '426bca34-828c-11ed-9fa7-005056921581',
 '8394d01c-5e3b-11ec-803b-005056921581',
 'dbfc8918-e272-

In [208]:
vector_template = np.zeros(len(total_components) * len(total_component_properties))

missing_components = []
missing_properties = []
def convertDataToVectorTest(components, components_propeties_names, components_propetries_values):
    result_vector = np.zeros(len(total_components) * len(total_component_properties))
    result_vector = np.array([np.float64(-9999999.0) for x in result_vector])
    
    for component_index, component_name in enumerate(components):

        try:
            # Индекс в векторе конкретного компонента
            component_vector_index = total_components.index(component_name)
        except:
            print(f"Component {component_name} not in TRAIN DATA!")
            missing_components.append(component_name)
        
        for component_property_index, component_propetry in enumerate(components_propeties_names[component_index]):
            try:
                # Индекс в векторе конкретного свойства компонента
                property_vector_index = total_component_properties.index(component_propetry)
            
                inserting_index = int((component_vector_index *  len(total_component_properties)) + property_vector_index)
                result_vector[inserting_index] = components_propetries_values[component_index][component_property_index]

            except: 
                print(f"Property {component_property} is missing in TRAIN DATA!")
                missing_properties.append(component_propetry)
            
            
    return result_vector

In [209]:

for index in range(len(workdata_oil_ids)):
    current_id = workdata_oil_ids[index]
    current_oil_data = workdata.loc[workdata["blend_id"] == current_id]
    
    components_in_oil = list(set(current_oil_data["component_name"].to_list()))
    
    components_propetries_to_function = []
    components_values_to_function = []
    
    for component in components_in_oil:
        # Извлекаю данные
        current_component_oil_data = current_oil_data.loc[current_oil_data["component_name"] == component]
        components_properties = current_component_oil_data["component_property_param_title"].to_list()
    
        # Подготавливаю для вектора
        current_component_properties = []
        current_component_values = []
    
        # Забираю значения по свойствам
        for component_property in components_properties:
            current_property_component_oil_data = current_component_oil_data.loc[current_component_oil_data["component_property_param_title"] == component_property]
            value = current_property_component_oil_data["component_property_param_value"].fillna(-9999999.0).values[0]
            
            current_component_properties.append(component_property)
            current_component_values.append(value)
    
        # Добавляю по свойствам для функции векторизации
        components_values_to_function.append(current_component_values)
        components_propetries_to_function.append(current_component_properties)

    current_result_vector = convertDataToVectorTest(components_in_oil, components_propetries_to_function, components_values_to_function)

    X_test_vector.append(current_result_vector)
    Ids.append(current_id)

Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAIN DATA!
Property 5f1a9612-c626-4f94-8e98-6a103e4f07ef is missing in TRAI

In [210]:
list(set(missing_components))

['GA5251',
 'К-433',
 'KUSAPOUR 85',
 'ЯНОС VHVI-6',
 'Компонент масла 3 погон',
 'PAO-6 EM',
 'Syntolux V15',
 'Evooil ITD-LG 2200 (Lubrizol 8405A)',
 'PX VI219 (Viscoplex 8-219)',
 'ATF WS',
 'Kusacryl 21618',
 'Transfer LF 75W',
 'BT 1#',
 'PAO 100 (PetroChina)',
 'ПМА - 7/Д']

In [211]:
list(set(missing_properties))

['- CP',
 'f6cc205c-a44d-40e5-9462-2eab4a673e2a',
 '- CN',
 '94d89046-80c9-41a8-9d0c-d587d9ed18c2',
 'd037c9ef-a6e1-4328-832a-9db50bbb92af',
 '943ccc3a-58c3-4ef7-a41f-02aa4956aeb1',
 'd6eff30f-788e-48a2-b2bd-cbba188f4e63',
 '24acf244-3846-49a7-be19-ebbff58fde91',
 '2bf4e562-53bd-4e6f-9f83-c30a892170e1',
 'П-ль преломления',
 '085c81d0-1885-4560-a4d3-08f9c0d108e2',
 '928fc6d7-374c-45c9-99bd-39de63a3f140',
 '00f815e2-e93a-4bac-9e2b-9733d77ab1ea',
 'b38cd6d1-cc71-4771-9d9c-7b37ae274fe9',
 '76287f81-2806-481a-8dc6-7274a761e2f9',
 '161776ad-2131-4557-a1f2-995bacb90bf0',
 '59a51599-1085-4a3e-9623-0a819b722db3',
 '52178b55-5b8f-48c4-8e8c-021f1facf40a',
 'a8bc9c11-42cb-4452-8cc8-8f96eb3d7705',
 '7fa1099c-cc3d-4bc2-ab0b-daa4fce8002e',
 '7fde038e-937b-4420-a4de-daf51b2ab54a',
 '50d7e65f-df1c-4e73-8961-9608846cf1d3',
 '834c8f63-4212-4e7e-b28e-2c79a42eb52f',
 '049e5cb9-9c20-438c-9ef2-96870712a0be',
 '- CA',
 'ebe9475f-b29f-4ffd-86af-a16a8321f1ee',
 '53d6b944-dbbd-40ee-a5b3-89bd0c983ab9',
 'Анилин.

In [212]:
len(list(set(missing_components)))

15

In [213]:
len(list(set(missing_properties)))

31

In [214]:
len(Ids)

138

In [215]:
best_predict = rf_2000_md_500.predict(X_test_vector)

In [216]:
submit = pd.DataFrame(Ids, columns=["blend_id"])

In [217]:
submit["result"] = best_predict

In [218]:
submit.to_csv("5th_rf2000.csv", encoding="utf-8", index=False)

In [219]:
submit

,blend_id,result
0,d272c9a6-3332-11ed-9685-005056921581,16148.403941
1,dea007ac-a065-11ed-9f24-005056921581,10007.697151
2,14efbc8a-9090-11ec-83a1-005056921581,10423.886262
3,e48772c8-4e07-11ed-9a6d-005056921581,15659.046993
4,72b7aafa-d791-11ed-8145-005056921581,10324.304214
...,...,...
133,994a200e-853c-11ee-9d2f-005056921581,11253.860864
134,eb25ecae-7886-11ee-9d5b-005056921581,18556.186402
135,e63f290a-fe22-11ed-8421-005056921581,17337.683973
136,f7ff118c-eb17-11ed-9974-005056921581,15845.069432
